In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib as mpl
from matplotlib import rc
# rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':10})
rc('mathtext',fontset='cm')
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}
import matplotlib.gridspec as gridspec

In [4]:
import numpy as np
import random as rn

In [4]:
n = 40
Xs = np.random.randn(150,10)
rs = Xs.sum(axis=1)
rs /= rs.std()

In [17]:
Xs1 = Xs[rs>=0]; r1 = rs[rs>=0]
Xs2 = Xs[rs<0]; r2 = rs[rs<0]

In [6]:
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax1,ax2 = ax
axcb = fig.add_axes([0.123,0.05,0.355,0.05])
ax1.set_aspect('equal')
ax2.set_aspect('equal')

c = ax1.scatter(Xs[:,0],Xs[:,1],c=rs,cmap='seismic')
cb = fig.colorbar(c,cax=axcb,use_gridspec=True,orientation='horizontal')
cb.set_label('$r$')

c1 = ax2.scatter(Xs1[:,0],Xs1[:,1],marker='+',label='$q(x) = \\alpha$')
c2 = ax2.scatter(Xs2[:,0],Xs2[:,1],marker='_',alpha=0.8,label='$q(x) = -\\alpha$')
ax2.scatter(Xs1[0,0],Xs1[0,1],c='w',alpha=0,label='$q(x) = 0$ ailleurs')
ax2.legend(ncol=3,labelspacing=0,mode='expand',bbox_to_anchor=(0, -1.05, 1, 1),frameon=False)

for ax in [ax1,ax2]:
    ax.axis(ymin=-3.2,ymax=3.2,xmin=-3.2,xmax=3.2)
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.annotate('$X_1$',xy=(0.2,2.8))
    ax.annotate('$X_2$',xy=(2.5,0.2))

ax1.set_title('a) Observations du marché')
ax2.set_title('b) Fonction de décision "dictionnaire"')    

# plt.savefig('fig/demo_1.pdf',bbox_inches='tight',pad_inches=0)
#


In [321]:
import cvxpy as cvx
from scipy.spatial.distance import pdist,cdist,squareform

In [384]:
X = Xs[:,:3]
X[:,2] = 1

In [395]:
sigma = 0.1

In [396]:
K = np.exp(-squareform(pdist(X))**2/sigma)
K.shape

(150, 150)

In [397]:
def solveg(sigma,l,r):
    # Only valid for exponential utility u(r) = -exp(-r)
    K = np.exp(-squareform(pdist(Xs))**2/sigma)
    n,_ = K.shape
    a = cvx.Variable(n)
    ra = cvx.mul_elemwise(r,a)
    obj = -cvx.sum_entries(cvx.kl_div(a,1)) - l*cvx.quad_form(ra,K)
    prob = cvx.Problem(cvx.Maximize(obj))
    prob.solve(solver=cvx.SCS)
    a = a.value.A1
    return a

In [398]:
def phi(sigma,X_s):
    phi = np.exp(-cdist(X,X_s)**2/sigma).T
    return phi

In [417]:
a = solveg(1,0.01,rs)

In [418]:
delta = 150
xx1 = np.linspace(min(X[:,0]),max(X[:,0]),delta)
xx2 = np.linspace(min(X[:,1]),max(X[:,1]),delta)
X_s = np.array(np.meshgrid(xx1,xx2,[1])).T[0].reshape((delta*delta,3))

In [419]:
Z = np.inner(phi(sigma,X_s),a*rs).reshape((delta,delta))

In [420]:
plt.scatter(X[:,0],X[:,1],c=rs,cmap='seismic');
plt.contour(xx1,xx2,Z,10)

In [7]:
import cvxpy as cvx
from scipy.spatial.distance import pdist,cdist,squareform

In [8]:
X = Xs[:,:3]
X[:,2] = 1

In [9]:
sigma = 0.1

In [10]:
K = np.exp(-squareform(pdist(X))**2/sigma)
K.shape

(150, 150)

In [11]:
def solveg(sigma,l,r):
    # Only valid for exponential utility u(r) = -exp(-r)
    K = np.exp(-squareform(pdist(Xs))**2/sigma)
    n,_ = K.shape
    a = cvx.Variable(n)
    ra = cvx.mul_elemwise(r,a)
    obj = -cvx.sum_entries(cvx.kl_div(a,1)) - l*cvx.quad_form(ra,K)
    prob = cvx.Problem(cvx.Maximize(obj))
    prob.solve(solver=cvx.SCS)
    a = a.value.A1
    return a

In [12]:
def phi(sigma,X_s):
    phi = np.exp(-cdist(X,X_s)**2/sigma).T
    return phi

In [13]:
a = solveg(1,0.01,rs)

In [52]:
delta = 150
xx1 = np.linspace(min(X[:,0]),max(X[:,0]),delta)
xx2 = np.linspace(min(X[:,1]),max(X[:,1]),delta)
X_s = np.array(np.meshgrid(xx1,xx2,[1])).T[0].reshape((delta*delta,3))

In [15]:
Z = np.inner(phi(sigma,X_s),a*rs).reshape((delta,delta))

In [16]:
plt.scatter(X[:,0],X[:,1],c=rs,cmap='seismic');
plt.contour(xx1,xx2,Z,10)

In [70]:
from helper import solve

In [90]:
n = 100
Xs = np.random.randn(150,3)
Xs[:,1] *= 10
rs = Xs.sum(axis=1)
rs /= rs.std()
Xs[:,2] = 1

In [91]:
Xs1 = Xs[rs>=0]; r1 = rs[rs>=0]
Xs2 = Xs[rs<0]; r2 = rs[rs<0]

In [103]:
q = cvx.Variable(3)
obj = cvx.Minimize(cvx.sum_squares(rs - Xs*q) + 10*cvx.norm(q)**2)
prob = cvx.Problem(obj)
prob.solve()
q_ls = q.value.A1

In [104]:
q_ls

array([ 0.10128766,  0.10618704,  0.00913582])

In [105]:
delta = 150
xx1 = np.linspace(Xs[:,0].min(),Xs[:,0].max(),delta)
xx2 = np.linspace(Xs[:,1].min(),Xs[:,1].max(),delta)
xx,yy = np.meshgrid(xx1,xx2)
z_ls = q_ls[0]*xx + q_ls[1]*yy + q_ls[2]


In [106]:
plt.scatter(Xs[:,0],Xs[:,1],c=rs,cmap='seismic')
cont = plt.contour(xx,yy,z_ls)
plt.clabel(cont)

<a list of 7 text.Text objects>

In [82]:
q_a = (Xs*rs[:,None]).mean(axis=0)
q_a

array([ 0.44193285,  0.62888695, -0.09663898])

In [98]:
z_a = q_a[0]*xx + q_a[1]*yy + q_a[2]
plt.scatter(Xs[:,0],Xs[:,1],c=rs,cmap='seismic')
cont = plt.contour(xx,yy,z_a)
plt.clabel(cont);


In [76]:
Xs*rs

ValueError: operands could not be broadcast together with shapes (150,3) (150,) 

In [269]:
from matplotlib import gridspec

In [31]:
import helper as he
from cd.model import utility as ut

In [139]:
u = ut.ExpUtility(1)

In [217]:
X = np.array([[-1.5, -0.5],
              [-0.5, -0.5],
              [-0.5, -1.5],
              [ 0.5,  0.5],
              [ 0.5,  1.5],
              [ 1.5,  0.5],
              [ 0.0 ,  0.5 ]])
r = np.array([-1,-2,-1,1,2,2,-2])
Xb = np.column_stack((X,np.ones(len(r))))

X2 = X[:-1]
r2 = r[:-1]
Xb2 = Xb[:-1]

In [260]:
lim = 2
xx1 = np.linspace(-lim,lim,100)
xx2 = np.linspace(-lim,lim,100)
xx1,xx2 = np.meshgrid(xx1,xx2)
zz1 = bias + q1*xx1 + q2*xx2
zz2 = bias2 + q12*xx1 + q22*xx2

In [0]:
fig, (ax1,ax2) = plt.subplots(2,1,subplot_kw={'aspect':1},figsize=(5,10))
fig.subplots_adjust(hspace=0.1)
titles = ['b) Ensemble d\'entraînement augmenté','a) Ensemble d\'entraînement original']
for ax,x,xb,rr,title in zip([ax2,ax1],[X,X2],[Xb,Xb2],[r,r2],titles):
    ax.scatter(*x[rr<0].T,color='C0',s=50*np.abs(rr[rr<0]),label='Rendement négatif')
    ax.scatter(*x[rr>0].T,color='C3',s=50*np.abs(rr[rr>0]),label='Rendement positif')
    [q1,q2,bias] = he.solve(xb*rr[:,None],u=u,lamb=0.5)[0]
    z = bias + q1*xx1 + q2*xx2
    cont = ax.contour(xx1,xx2,z,[-1,0,1],colors=('C0','k','C3'),linestyles=('--','--','--'))
    ax.clabel(cont,fmt={-1:'$\hat q(x)=-1$',0:'$\hat q(x)=0$',1:'$\hat q(x)=1$'})

    ax.axis([-lim,lim,-lim,lim])
    ax.spines['left'].set_position('zero')
    ax.spines['left'].set_alpha(0.5)
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('zero')
    ax.spines['bottom'].set_alpha(0.5)
    ax.spines['top'].set_color('none')

    ax.set_title(title)

ax2.legend(ncol=2,mode='expand',bbox_to_anchor=(0., -0.15, 1., .102),borderaxespad=0.)
plt.savefig('fig/linker.pdf',bbox_inches='tight',pad_inches=0)


In [1]:
from scipy.spatial.distance import pdist,cdist,squareform

In [5]:
def center_axis(ax,lim=1):
    ax.axis([-lim,lim,-lim,lim])
    ax.spines['left'].set_position('zero')
    ax.spines['left'].set_alpha(0.5)
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('zero')
    ax.spines['bottom'].set_alpha(0.5)
    ax.spines['top'].set_color('none')

In [6]:
Xp = [[-1.5,-1.5],
      [-1.0,-0.5],
      [-0.6,-1.3],
      [1.6,-1],
      [-0.8,1],
      [-1.6,-0.1]]
rp = [2,1,1.3,0.3,0.2,1]

Xn = [[0.5,0.4],
      [1.1,1.7],
      [1.8,1],
      [0.6,1.25],
      [0.2,-0.8],
      [0.9,-1.4],
      [-1.4,1.5]]
rn = [-1.0,-1.3,-0.8,-2,-0.2,-0.25,-0.5]

Xp = np.array(Xp)
rp = np.array(rp)
Xn = np.array(Xn)
rn = np.array(rn)


In [53]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,s=40*np.abs(rp),color='C3',label='Rendement positif')
ax.scatter(*Xn.T,s=40*np.abs(rn),color='C0',label='Rendement négatif')
center_axis(ax,lim=2)
ax.annotate('Prix du pétrole (acroissement %)',xy=(0.35,0.1))
ax.annotate('Taux de chômage\n(acroissement %)',xy=(-1.,1.7))
ax.legend(loc='lower right',bbox_to_anchor=(0., -0.012, 1., .102),borderaxespad=0.)

plt.savefig('fig/pres/pres1_fr.pdf',bbox_inches='tight',pad_inches=0)

In [67]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,color='C3',s=100,marker='+',label="Inv. positif ($q(x) = +\\alpha$)")
ax.scatter(*Xn.T,color='C0',s=100,marker='_',label="Inv.  négatif ($q(x) = -\\alpha$)")
ax.scatter([0],y=[0],marker=' ',label='Inv. nul partout ailleurs ($q(x)=0$)')
center_axis(ax,lim=2)
ax.annotate('Prix du pétrole (acroissement %)',xy=(0.35,0.1))
ax.annotate('Taux de chômage\n(acroissement %)',xy=(-1.,1.7))
ax.legend(loc='lower right',bbox_to_anchor=(0, -0.02, 1.05, .102),borderaxespad=0.);

plt.savefig('fig/pres/pres2_fr.pdf',bbox_inches='tight',pad_inches=0)



In [68]:
X = np.concatenate((Xp,Xn),axis=0)
X = np.insert(X,2,1,axis=1)
r = np.concatenate((rp,rn))

In [75]:
import helper as he
from cd.model import utility as ut
u = ut.ExpUtility(1)
lim = 2

In [76]:
[q1,q2,bias] = he.solve(X*r[:,None],u=u,lamb=0.3)[0]
xx = np.linspace(-lim,lim,100)
yy = np.linspace(-lim,lim,100)
xx,yy = np.meshgrid(xx,yy)
zz = bias + q1*xx + q2*yy

In [82]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,s=40*np.abs(rp),color='C3',label='Positive return')
ax.scatter(*Xn.T,s=40*np.abs(rn),color='C0',label='Negative return')
cont = ax.contour(xx,yy,zz,[-1,0,1],colors=('C0','k','C3'),linestyles='--')
ax.clabel(cont,fmt={-1:'$\hat q(x)=-100\%$',0:'$\hat q(x)=0$',1:'$\hat q(x)=100\%$'})
center_axis(ax,lim=2)
ax.annotate('Prix du pétrole (acroissement %)',xy=(0.35,0.1))
ax.annotate('Taux de chômage\n(acroissement %)',xy=(-1.,1.7))
# ax.legend(loc='lower right',bbox_to_anchor=(0., -0.012, 1., .102),borderaxespad=0.)

plt.savefig('fig/pres/pres3_fr.pdf',bbox_inches='tight',pad_inches=0)
            

In [118]:
Xp = [[-1.,-1.],
      [1.,1.]]
Xn = [[-1.,1.],
      [1.,-1.]]
rp = [1,1]
rn = [-1,-1]
Xp = np.array(Xp)
Xn = np.array(Xn)
rp = np.array(rp)
rn = np.array(rn)

In [130]:
fig,[ax,ax2] = plt.subplots(2,1,subplot_kw={'aspect':1},figsize=(6,12))
fig.subplots_adjust(hspace=0.1)
ax.set_title('a) Problème XOR')
ax.scatter(*Xp.T,s=100,color='C3',label='Rendement $+1$')
ax.scatter(*Xn.T,s=100,color='C0',label='Rendement $-1$')
center_axis(ax,lim=2)
ax.annotate('$X_1$',xy=(1.75,0.1))
ax.annotate('$X_2$',xy=(0.1,1.8))
ax.legend(loc='lower right',bbox_to_anchor=(0., -0.012, 1., .102),borderaxespad=0.)

ax2.set_title('b) Solution avec transformation quadratique')
ax2.scatter(*Xp.T,s=100,color='C3',label='Rendement: $+1$')
ax2.scatter(*Xn.T,s=100,color='C0',label='Rendement: $-1$')
cont = ax2.contour(xx,yy,zz,[-1,0,1],colors=('C0','k','C3'),linestyles='--')
ax2.clabel(cont,fmt={-1:'$\hat q(x)=-1$',0:'$\hat q(x)=0$',1:'$\hat q(x)=1$'})
center_axis(ax2,lim=2)
ax2.annotate('$X_1$',xy=(1.75,0.1))
ax2.annotate('$X_2$',xy=(0.1,1.8))
ax2.legend(loc='lower right',bbox_to_anchor=(0., -0.012, 1., .102),borderaxespad=0.)

plt.savefig('fig/pres/pres5_fr.pdf',bbox_inches='tight',pad_inches=0)

In [97]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,s=100,color='C3',label='Rendement positif')
ax.scatter(*Xn.T,s=100,color='C0',label='Rendement négatif')
# cont = ax.contour(xx,yy,zz,[0],colors=('k',),linestyles='--')
center_axis(ax,lim=2)
ax.annotate('$X_1$',xy=(1.75,0.1))
ax.annotate('$X_2$',xy=(0.1,1.8))
ax.legend(loc='lower right',bbox_to_anchor=(0., -0.012, 1., .102),borderaxespad=0.)


plt.savefig('fig/pres/pres4_fr.pdf',bbox_inches='tight',pad_inches=0)

In [98]:
X = np.concatenate((Xp,Xn),axis=0)
# X = np.insert(X,2,1,axis=1)
r = np.concatenate((rp,rn))

In [99]:
def quad1(x):
    return np.array([x[0]**2,x[1]**2,x[0]*x[1],x[0],x[1],1])

In [100]:
phiX = np.apply_along_axis(quad1,1,X)

In [103]:
q = he.solve(phiX*r[:,None],u=u,lamb=0.1)[0]
xx = np.linspace(-lim,lim,100)
yy = np.linspace(-lim,lim,100)
xx,yy = np.meshgrid(xx,yy)
zz = np.apply_along_axis(quad1,0,[xx,yy]).T @ q

In [104]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,s=100,color='C3',label='Positive return')
ax.scatter(*Xn.T,s=100,color='C0',label='Negative return')
cont = ax.contour(xx,yy,zz,[-1,0,1],colors=('C0','k','C3'),linestyles='--')
ax.clabel(cont,fmt={-1:'$\hat q(x)=-1$',0:'$\hat q(x)=0$',1:'$\hat q(x)=1$'})
center_axis(ax,lim=2)
ax.annotate('$X_1$',xy=(1.75,0.1))
ax.annotate('$X_2$',xy=(0.1,1.8))
ax.legend(ncol=2,mode='expand',loc='lower left',bbox_to_anchor=(0., -0.08, 1., .102),borderaxespad=0.)

# plt.savefig('fig/pres/pres5.pdf',bbox_inches='tight',pad_inches=0)

In [131]:
Xp = [[-1.5,-1.5],
      [-1.0,-0.5],
      [-0.6,-1.3],
      [1.6,-1],
      [-0.8,1],
      [-1.6,-0.1]]
rp = [2,1,1.3,0.3,0.2,1]

Xn = [[0.5,0.4],
      [1.1,1.7],
      [1.8,1],
      [0.6,1.25],
      [0.2,-0.8],
      [0.9,-1.4],
      [-1.4,1.5]]
rn = [-1.0,-1.3,-0.8,-2,-0.2,-0.25,-0.5]

Xp = np.array(Xp)
rp = np.array(rp)
Xn = np.array(Xn)
rn = np.array(rn)


In [132]:
X = np.concatenate((Xp,Xn),axis=0)
# X = np.insert(X,2,1,axis=1)
r = np.concatenate((rp,rn))

In [133]:
phiX = np.apply_along_axis(quad1,1,X)

In [134]:
q = he.solve(phiX*r[:,None],u=ut.ExpUtility(0.8),lamb=0.1)[0]
xx = np.linspace(-lim,lim,100)
yy = np.linspace(-lim,lim,100)
xx,yy = np.meshgrid(xx,yy)
zz = np.apply_along_axis(quad1,0,[xx,yy]).T @ q

In [152]:
fig,[ax,ax2] = plt.subplots(2,1,subplot_kw={'aspect':1},figsize=(6,12))
fig.subplots_adjust(hspace=0.1)
ax.set_title('a) Noyau quadratique')
ax.scatter(*Xp.T,s=80*np.abs(rp),color='C3',label='Positive return')
ax.scatter(*Xn.T,s=80*np.abs(rn),color='C0',label='Negative return')
cont = ax.contour(xx,yy,zz,[-1,-0.5,0,0.5,1],colors=('C0','C0','k','C3','C3'),linestyles=('--'))
ax.clabel(cont,fmt={-1   : '$\hat q(x)=-1$',
                    -0.5 : '$\hat q(x)=-1/2$',
                    0    : '$\hat q(x)=0$',
                    0.5  : '$\hat q(x)=1/2$',
                    1    : '$\hat q(x)=1$'})
center_axis(ax,lim=2)
ax.annotate('Prix du pétrole (acroissement %)',xy=(0.35,0.1))
ax.annotate('Taux de chômage\n(acroissement %)',xy=(-1.,1.7))


ax2.set_title('b) Noyau gaussien')
ax2.scatter(*Xp.T,s=80*np.abs(rp),color='C3',label='Positive return')
ax2.scatter(*Xn.T,s=80*np.abs(rn),color='C0',label='Negative return')
cont = ax2.contour(xx,yy,Z,[-1,-0.5,0,0.5,1],colors=('C0','C0','k','C3','C3'),linestyles=('--'))
ax2.clabel(cont,fmt={-1   : '$\hat q(x)=-1$',
                    -0.5 : '$\hat q(x)=-1/2$',
                    0    : '$\hat q(x)=0$',
                    0.5  : '$\hat q(x)=1/2$',
                    1    : '$\hat q(x)=1$'})
center_axis(ax2,lim=2)
ax2.annotate('Prix du pétrole (acroissement %)',xy=(0.35,0.1))
ax2.annotate('Taux de chômage\n(acroissement %)',xy=(-1.,1.7))





plt.savefig('fig/pres/pres10_fr.pdf',bbox_inches='tight',pad_inches=0)

In [135]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,s=80*np.abs(rp),color='C3',label='Positive return')
ax.scatter(*Xn.T,s=80*np.abs(rn),color='C0',label='Negative return')
cont = ax.contour(xx,yy,zz,[-1,-0.5,0,0.5,1],colors=('C0','C0','k','C3','C3'),linestyles=('--'))
ax.clabel(cont,fmt={-1   : '$\hat q(x)=-1$',
                    -0.5 : '$\hat q(x)=-1/2$',
                    0    : '$\hat q(x)=0$',
                    0.5  : '$\hat q(x)=1/2$',
                    1    : '$\hat q(x)=1$'})
center_axis(ax,lim=2)
ax.annotate('Oil price (increase %)',xy=(0.8,0.1))
ax.annotate('US unemployment\n(increase %)',xy=(-1.3,1.7))
ax.legend(ncol=2,mode='expand',loc='lower left',bbox_to_anchor=(0., -0.08, 1., .102),borderaxespad=0.)

# plt.savefig('fig/pres/pres6.pdf',bbox_inches='tight',pad_inches=0)

In [138]:
X = np.concatenate((Xp,Xn),axis=0)
X = np.insert(X,2,1,axis=1)
r = np.concatenate((rp,rn))

In [139]:
sigma = 0.5
def gauss_K(X1,X2=None):
    if X2 is None:
        K = np.exp(-squareform(pdist(X1))**2/sigma)
        return K
    else:
        K = np.exp(-cdist(X1,X2)**2/sigma).T
        return K

In [141]:
K = gauss_K(X)

In [143]:
a = he.solve_k(K,r,u,lamb=0.1)*8

In [144]:
delta = 100
xx1 = np.linspace(-lim,lim,delta)
xx2 = np.linspace(-lim,lim,delta)
X_s = np.array(np.meshgrid(xx1,xx2,[1])).T[0].reshape((100*100,3))
phi = gauss_K(X,X_s)
Z = np.inner(phi,a).reshape((delta,delta)).T

In [145]:
fig,ax = plt.subplots(1,1,subplot_kw={'aspect':1},figsize=(6,6))
ax.scatter(*Xp.T,s=80*np.abs(rp),color='C3',label='Positive return')
ax.scatter(*Xn.T,s=80*np.abs(rn),color='C0',label='Negative return')
cont = ax.contour(xx,yy,Z,[-1,-0.5,0,0.5,1],colors=('C0','C0','k','C3','C3'),linestyles=('--'))
ax.clabel(cont,fmt={-1   : '$\hat q(x)=-1$',
                    -0.5 : '$\hat q(x)=-1/2$',
                    0    : '$\hat q(x)=0$',
                    0.5  : '$\hat q(x)=1/2$',
                    1    : '$\hat q(x)=1$'})
center_axis(ax,lim=2)
ax.annotate('Oil price (increase %)',xy=(0.8,0.1))
ax.annotate('US unemployment\n(increase %)',xy=(-1.3,1.7))
ax.legend(ncol=2,mode='expand',loc='lower left',bbox_to_anchor=(0., -0.08, 1., .102),borderaxespad=0.)

# plt.savefig('fig/pres/pres7.pdf',bbox_inches='tight',pad_inches=0)

array([[-2.        , -2.        ,  1.        ],
       [-2.        , -1.95959596,  1.        ],
       [-2.        , -1.91919192,  1.        ],
       ..., 
       [ 2.        ,  1.91919192,  1.        ],
       [ 2.        ,  1.95959596,  1.        ],
       [ 2.        ,  2.        ,  1.        ]])